In [1]:
GYRO = False
SEQUENCE_LENGTH = 3
SEQUENCE_OVERLAP = 2
BATCH_SIZE = 10
EPOCHS = 50
MODEL_NAME = f"position_epochs:{EPOCHS}_batch:{BATCH_SIZE}_gyro:{GYRO}_window:{SEQUENCE_LENGTH}_overlap:{SEQUENCE_OVERLAP}"

### This code trains a classifier that returns what position the activity is in, eg "sitting_or_standing", "lying_down_left" etc, using only respiratory symptoms data


In [2]:
import file_tagger
import sequence_genrator
import tensorflow as tf
import numpy as np
from keras import layers, Sequential, models
from sklearn.model_selection import train_test_split

POSITIONS = {
    "sitting_or_standing",
    "lying_down_back",
    "lying_down_stomach",
    "lying_down_right",
    "lying_down_left"
}


TASK3_ACTIVITIES = {
    "sitting&coughing",
    "sitting&hyperventilating",
    "sitting&normal_breathing",
    
    "standing&coughing",
    "standing&hyperventilating",
    "standing&normal_breathing",
    
    "lying_down_back&coughing",
    "lying_down_back&hyperventilating",
    "lying_down_back&normal_breathing",
    
    "lying_down_stomach&coughing",
    "lying_down_stomach&hyperventilating",
    "lying_down_stomach&normal_breathing",
    
    "lying_down_right&coughing",
    "lying_down_right&hyperventilating",
    "lying_down_right&normal_breathing",
    
    "lying_down_left&coughing",
    "lying_down_left&hyperventilating",
    "lying_down_left&normal_breathing",

}

DATA_DIRECTORY = "./all_respeck"

2023-11-07 17:04:49.360834: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-07 17:04:49.386534: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-07 17:04:49.555540: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-07 17:04:49.555638: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-07 17:04:49.556905: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [3]:
# split data into training, dev, and test sets
def train_dev_test_split(data, labels, dev_size, test_size):
    # Split the data into training and temporary (dev + test) sets
    train_data, temp_data, train_labels, temp_labels = train_test_split(data, labels, test_size=(dev_size + test_size))
    
    # Split the temporary data into dev and test sets
    dev_data, test_data, dev_labels, test_labels = train_test_split(temp_data, temp_labels, 
                                                                 test_size=(test_size / (dev_size + test_size)))
    
    return train_data, dev_data, test_data, train_labels, dev_labels, test_labels

In [4]:
# generate training data
def generate_training_data(directory, sequence_length, overlap, gyro = GYRO): # if gyro is false, only accelerometer data is used

    tagged_data = []

    # group each csv file into their respective areas
    csv_dictionary = file_tagger.tag_directory(directory)

    # iterates through each activity
    for key in TASK3_ACTIVITIES:

        # iterates through each csv file for the activity 
        for csv_file in csv_dictionary[key]:
            if gyro:
                sequences = sequence_genrator.generate_sequences_from_file_with_gyroscope(directory + "/" + csv_file, sequence_length, overlap)
            else:
                sequences = sequence_genrator.generate_sequences_from_file_without_gyroscope(directory + "/" + csv_file, sequence_length, overlap)

            # iterate through each generated sequence
            for sequence in sequences:
                tagged_data.append((key, sequence))

    print ("there are " + str(len(tagged_data)) + " tagged sequences in the dataset")
    return tagged_data

In [5]:
#CNN MODEL
def train_model_CNN(input_data, labels_encoded, unique_labels, epochs, batch_size, validation_data):
    if GYRO:
        width = 6
    else:
        width = 3
    # Define the CNN model for your specific input shape
    model = Sequential([
        layers.Conv1D(32, 3, activation='relu', input_shape=(SEQUENCE_LENGTH*25, width)),
        layers.MaxPooling1D(2),
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Conv1D(128, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Dropout(0.5),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(len(unique_labels), activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the CNN model
    model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size, validation_data=validation_data)

    return model

In [6]:

def tag_data_with_position(tagged_data):
    tagged_data_with_position = []
    for tag, sequence in tagged_data:
        position = tag.split("&")[0]
        if position == "sitting" or position == "standing":
            position = "sitting_or_standing"
        tagged_data_with_position.append((position, sequence))
    return tagged_data_with_position

def generate_position_model(tagged_sequences):
    tagged_sequences_with_position = tag_data_with_position(tagged_sequences)
    # Combine all sequences and labels
    sequences = [sequence for _, sequence in tagged_sequences_with_position]
    labels = [label for label, _ in tagged_sequences_with_position]


    # encode labels to numbers
    sequences = np.array(sequences, dtype=np.float32)
    label_to_index = {label: idx for idx, label in enumerate(POSITIONS)}
    labels_encoded = [label_to_index[label] for label in labels]
    labels_encoded = np.array(labels_encoded)

    train_data, dev_data, test_data, train_labels, dev_labels, test_labels = train_dev_test_split(sequences, labels_encoded, dev_size=0.1, test_size=0.1) #10% dev, 10% test


    # train and save model (CHOOSE BETWEEN CNN AND LSTM)
    model = train_model_CNN(train_data, train_labels, POSITIONS, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(dev_data, dev_labels)) #batch_size, epochs


    # Evaluate the model on the test set
    test_loss, test_accuracy = model.evaluate(test_data, test_labels)
    print (f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")
    return model

In [7]:
tagged_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP, GYRO)
model = generate_position_model(tagged_sequences)

# Save the trained model
#model.save(f"models/Task2/{MODEL_NAME}_{test_accuracy}.keras")

there are 23159 tagged sequences in the dataset
Epoch 1/50
1853/1853 [==============================] - 5s 3ms/step - loss: 0.1625 - accuracy: 0.9630 - val_loss: 0.0903 - val_accuracy: 0.9745
Epoch 2/50
1853/1853 [==============================] - 4s 2ms/step - loss: 0.1039 - accuracy: 0.9716 - val_loss: 0.0771 - val_accuracy: 0.9763
Epoch 3/50
1853/1853 [==============================] - 5s 3ms/step - loss: 0.0907 - accuracy: 0.9737 - val_loss: 0.0873 - val_accuracy: 0.9719
Epoch 4/50
1853/1853 [==============================] - 5s 3ms/step - loss: 0.0800 - accuracy: 0.9750 - val_loss: 0.0619 - val_accuracy: 0.9788
Epoch 5/50
1853/1853 [==============================] - 5s 3ms/step - loss: 0.0776 - accuracy: 0.9755 - val_loss: 0.0744 - val_accuracy: 0.9767
Epoch 6/50
1853/1853 [==============================] - 5s 3ms/step - loss: 0.0796 - accuracy: 0.9752 - val_loss: 0.0516 - val_accuracy: 0.9840
Epoch 7/50
1853/1853 [==============================] - 5s 3ms/step - loss: 0.0714 - acc